# Eigen Portfolios
#### Authors: António Salomão, Carolina Domingues, Tim Gajewski
##### Credits: PMC© Quant Team, Fall 2021

### Imports

In [1]:
# Show multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# YFinance
import yfinance as yf

# Pytickersymbols
import pytickersymbols
from pytickersymbols import PyTickerSymbols

# Common imports
import pandas as pd
import numpy as np
from datetime import datetime

# Numpy
from numpy.linalg import eig

# PLotly
import plotly.express as px

# Pandas Datareader for Fama French Regresseion
import pandas_datareader.data as reader

#
import statsmodels.api as sn

#
import datetime

#
import quantstats as qs


### Data cleansing (Adjusted Close)

We'll drop columns that have "too many" NaN values.

In [2]:
def clean_df_by_nan(df, thresh, fill_na):
    # Assert
    assert isinstance(df, pd.DataFrame)
    # Avoid overwritting original dataframe
    df = df.copy()
    # Compute the fraction of missing values per column
    data_fractions = 1. - df.isnull().mean().sort_values(ascending=False)
    drop_list = sorted(list(data_fractions[data_fractions < thresh].index))
    # Drop problematic columns
    if drop_list:
        df = df.drop(drop_list, axis=1)
    # Fill missing values with the last value avail. (forward-fill)
    if fill_na:
        df = df.fillna(method="ffill")
        df = df.dropna(axis=0)
    
    return df

### Eigenportfolios

In [3]:
def eigenportfolio (startdate, enddate, ticker):    
    # Get US ticker symbols (SP500)
    stock_data = PyTickerSymbols()
    us_stocks = list(stock_data.get_stocks_by_index(ticker))
    us_ticker_symbols = [k["symbol"] for k in us_stocks]

    # YFinance
    us_hist_df = yf.download(tickers=us_ticker_symbols,
                             start=startdate,
                             end=enddate,
                             interval="1d")

    us_hist_df.index = pd.to_datetime(us_hist_df.index)


    # Removing NaN values and cleaning columns
    us_adjClose = clean_df_by_nan(df=us_hist_df["Adj Close"], thresh=0.90, fill_na=True)


    # Individual securities
    us_log_rets = np.log(us_adjClose) - np.log(us_adjClose.shift(1))
    us_log_rets = us_log_rets.dropna(axis=0)


    us_cov_log_rets = us_log_rets.cov()
    cov_fig = px.imshow(us_cov_log_rets, width=500)
    us_cov_log_rets.shape


    # Standardizing
    mu, sigma = us_log_rets.mean(), us_log_rets.std()
    us_log_rets_rescaled = (us_log_rets.sub(mu, axis=1)).div(sigma, axis=1)


    # Train-test split
    cutoff = 0.8
    n = len(us_log_rets_rescaled)
    X_train_raw, X_test_raw = us_log_rets[:int(n*cutoff)], us_log_rets[int(n*cutoff):]
    X_train_rescaled , X_test_rescaled = us_log_rets_rescaled[:int(n*cutoff)], us_log_rets_rescaled[int(n*cutoff):]


    X_train_cov = X_train_rescaled.cov()
    eigenvalues, eigenvectors = eig(X_train_cov)


    exp_var_ratios = eigenvalues / eigenvalues.sum()


    # Explained variace ratio by Top Factors
    exp_var_ratios_fig = px.bar(x=exp_var_ratios[:20],
                                labels={"x": "Explained Variance", "y": "Component"},
                                height=400,
                                width=800)

    # Explained cumulative variance
    exp_var_cumul = np.cumsum(exp_var_ratios)
    exp_var_cumul_fig = px.area(x=range(1, exp_var_cumul.shape[0] + 1),
                                y=exp_var_cumul,
                                labels={"x": "# Components", "y": "Explained Variance"},
                                height=400,
                                width=800)


    eigenPortfolios = list(map(lambda x: x / sum(x) , eigenvectors))
    eigenPortfolios_df = pd.DataFrame(eigenPortfolios, columns=X_train_raw.columns)

    for i in range(5):

        fig_i = px.bar(eigenPortfolios_df.iloc[i].to_frame(),
                       title=f"Eigen Portfolio #{i} | Eigenvalue: {eigenvalues[i]:.2f} | Explained Variance: {exp_var_ratios[i]:.2f}",
                       labels={"index":"Ticker", "value": "Weight"},
                       width=1000,
                       height=500)


    def get_sharpe_ratio(x):
        assert len(x.shape) == 1
        x = x.copy()
        ann_ret = x.mean() * 252.
        ann_std = x.std() * np.sqrt(252.)
        sr = ann_ret / ann_std
        return ann_ret, ann_std, sr

    my_eigenports = {"Eigen Portfolio #": [],
                               "Weights": [],
                                "Return": [],
                            "Volatility": [],
                          "Sharpe Ratio": []}

    # Compute Sharpe Ratio for every eigen portfolio
    for i in range(len(eigenPortfolios_df)):
        eigenPort_i = eigenPortfolios_df.iloc[i].to_numpy()
        rets_i = np.dot(X_train_raw, eigenPort_i)
        ret, vol, sr = get_sharpe_ratio(rets_i)
        my_eigenports["Eigen Portfolio #"].append(str(i))          
        my_eigenports["Weights"].append(eigenPort_i)
        my_eigenports["Return"].append(ret)
        my_eigenports["Volatility"].append(vol)
        my_eigenports["Sharpe Ratio"].append(sr)


    eigenports_sr_df = pd.DataFrame(my_eigenports["Sharpe Ratio"], columns=["Sharpe Ratio"])
    eigenports_sr_sorted_df = eigenports_sr_df.sort_values(by="Sharpe Ratio", ascending=False)

    eigenports_sr_fig1 = px.bar(x=list(map(str, eigenports_sr_df.index.to_numpy().squeeze())),
                                y=eigenports_sr_df.to_numpy().squeeze(),
                                labels = {"x": "Eigenportfolio #", "y": "Sharpe Ratio"},
                                title="Eigen Porfolios sorted by Eigenvalues (explained variance)",
                                width=1000,
                                height=500)

    eigenports_sr_fig2 = px.bar(x=list(map(str, eigenports_sr_sorted_df.index.to_numpy().squeeze())),
                                y=eigenports_sr_sorted_df.to_numpy().squeeze(),
                                labels={"x": "Eigenportfolio #", "y": "Sharpe Ratio"},
                                title="Eigen Porfolios sorted by Sharpe Ratio",
                                width=1000,
                                height=500)


    def get_eigen_rets(test_df, n_eigen_portfolios):

        # Copy
        test = test_df.copy()
        eigenports_sr = eigenports_sr_df.copy()
        eigenports_dict = my_eigenports.copy()

        # Sort by sharpe ratio
        eigenports_sr = eigenports_sr.sort_values(by="Sharpe Ratio", ascending=False)

        # Eigen portfolios
        eigen_ports = eigenports_sr.index[:n_eigen_portfolios]
        eigen_weights = [eigenports_dict["Weights"][i] for i in eigen_ports]

        # Returns
        eigen_rets = np.array(list(map(lambda n: np.dot(test, n), eigen_weights)))
        eigen_df = pd.DataFrame(eigen_rets.T, 
                                columns=list(map(str, eigen_ports)), 
                                index=test.index)

        # Equal-weighted eigen portfolio
        ew_eigen_rets = (eigen_rets.T * (1./eigen_rets.T.shape[1])).sum(axis=1)
        ew_eigen_df = pd.DataFrame(ew_eigen_rets,
                                   columns=["EW-Eigen"],
                                   index=test.index)
        # Equal-weighted SP500
        ew_sp_rets = (test * (1./test.shape[1])).sum(axis=1)
        ew_sp_df = pd.DataFrame(ew_sp_rets,
                                columns=["EW-SP500"],
                                index=test.index)

        all_eigen_df = eigen_df.join([ew_eigen_df, ew_sp_df])

        return all_eigen_df

    def get_eigen_stats(rets_df):

        rets_df = rets_df.copy()

        # Compute sharpe ratio
        sr = np.array(rets_df.apply(get_sharpe_ratio, axis=0).T)

        stats = pd.DataFrame(data=sr,
                             columns=["Return", "Volatility", "SR"],
                             index=rets_df.columns).sort_values(by="SR", ascending=False)
        return stats


    ew_eigen_rets_train = list(map(lambda eig_n: get_eigen_rets(X_train_raw, eig_n), range(1, 80)))
    ew_eigen_stats_train = list(map(lambda eig: get_eigen_stats(eig).loc["EW-Eigen"].values, ew_eigen_rets_train))
    ew_eigen_stats_train_df = pd.DataFrame(ew_eigen_stats_train, columns=["Return", "Volatility", "SR"])

    ew_eigen_rets_test = list(map(lambda eig_n: get_eigen_rets(X_test_raw, eig_n), range(1, 80)))
    ew_eigen_stats_test = list(map(lambda eig: get_eigen_stats(eig).loc["EW-Eigen"].values, ew_eigen_rets_test))
    ew_eigen_stats_test_df = pd.DataFrame(ew_eigen_stats_test, columns=["Return", "Volatility", "SR"])


    ew_ret_comparison = pd.DataFrame(data=[ew_eigen_stats_train_df["Return"], ew_eigen_stats_test_df["Return"]],
                                     index=["Train", "Test"]).T

    ew_vol_comparison = pd.DataFrame(data=[ew_eigen_stats_train_df["Volatility"], ew_eigen_stats_test_df["Volatility"]],
                                     index=["Train", "Test"]).T

    ew_sr_comparison = pd.DataFrame(data=[ew_eigen_stats_train_df["SR"], ew_eigen_stats_test_df["SR"]],
                                    index=["Train", "Test"]).T

    ret_fig = px.line(ew_ret_comparison, 
                      title="Return", 
                      labels={"index" : "EW-Eigen #"})

    vol_fig = px.line(ew_vol_comparison, 
                      title="Volatility", 
                      labels={"index" : "EW-Eigen #"})

    sr_fig = px.line(ew_sr_comparison, 
                     title="Sharpe Ratio", 
                     labels={"index" : "EW-Eigen #"})

    ew_ret_comparison.sort_values(by="Train", ascending=False).head(5)
    ew_vol_comparison.sort_values(by="Train", ascending=True).head(5)
    ew_sr_comparison.sort_values(by="Train", ascending=False).head(5)


    eigen_train_rets = get_eigen_rets(test_df=X_train_raw, n_eigen_portfolios=14)

    # Sharpe Ratios
    eigen_train_sr = get_eigen_stats(eigen_train_rets)
    eigen_train_sr.head(5)

    # Cumulative Returns
    eigen_train_fig = px.line(1. + eigen_train_rets.cumsum(),
                              labels={"value" : "Cumulative Return"},
                              title=f"Top Eigen Portfolios Comparison vs EW Eigen Portfolio vs EW SP500",
                              width=1000,
                              height=400)

    eigen_train_sr.loc[["EW-Eigen", "EW-SP500"]]


    eigen_test_rets = get_eigen_rets(test_df=X_test_raw, 
                                     n_eigen_portfolios=14)

    # Sharpe ratios
    eigen_test_sr = get_eigen_stats(eigen_test_rets)
    eigen_test_sr.head(5)

    # Cumualtive returns
    eigen_test_fig = px.line(1. + eigen_test_rets.cumsum(),
                             labels={"value" : "Cumulative Return"},
                             title=f"Top Eigen Portfolios Comparison vs EW Eigen Portfolio vs EW SP500",
                             width=1000,
                             height=400)

    eigen_test_sr.loc[["EW-Eigen", "EW-SP500"]]

    return [get_eigen_rets(test_df=X_test_raw, n_eigen_portfolios=14)['EW-Eigen'], get_eigen_rets(test_df=X_test_raw, n_eigen_portfolios=14)['EW-SP500']]


#### Explain Returns (Fama French)

In [4]:
#Merge Data Frames
def merge_df(ret_mtl, factors):
    merge=pd.merge(ret_mtl,factors,on='Date')

    # Factors from % to absolut
    merge[['Mkt-RF','SMB','HML','RF']]=merge[['Mkt-RF','SMB','HML','RF']]/100

    # Excess Returns
    merge['EW-RF']=merge[list(merge.columns)[0]] - merge['RF']
    return merge

In [5]:
# Regression
def regression_fama_french(merged_factor_df, portfolio_title):
    y_1=merged_factor_df['EW-RF']
    X_1=merged_factor_df[['Mkt-RF', 'SMB', 'HML']]

    X_sn_1= sn.add_constant(X_1)
    
    # Regression statistics
    model_1= sn.OLS(y_1,X_sn_1)
    results_1=model_1.fit()
    print(portfolio_title)
    print(results_1.summary())
    print('\n')

 

In [6]:
# Monthly returns for Fama French
def fama_french(portfolio):
    
    ew_eig_ret_mtl_list=[]
    ew_SP500_ret_mtl_list=[]
    factors_list=[]
    ew_eig_ret_list=[]
    for i in range(0, len(eigenportfolios)):
        ew_eigen_ret=np.exp(eigenportfolios[i][0])
        ew_SP500_ret=np.exp(eigenportfolios[i][1])

        # For Quant Stats
        ew_eig_ret_list.append(ew_eigen_ret)

        ew_eigen_ret_mtl=ew_eigen_ret.resample('M').agg(lambda x: (x).prod()-1)
        ew_SP500_ret_mtl=ew_SP500_ret.resample('M').agg(lambda x: (x).prod()-1)

        #Fama French Factors
        start=ew_eigen_ret_mtl.index[0]
        end=ew_eigen_ret_mtl.index[ew_eigen_ret_mtl.size-1]
        factors=reader.DataReader('F-F_Research_Data_Factors','famafrench',start,end)[0]

        # Formating Month Dates
        ew_eigen_ret_mtl.index=factors.index
        ew_SP500_ret_mtl.index=factors.index

        ew_eig_ret_mtl_list.append(ew_eigen_ret_mtl)
        ew_SP500_ret_mtl_list.append(ew_SP500_ret_mtl)
        factors_list.append(factors)
    
    # Regression statistics
    for i in range(0,len(ew_eig_ret_mtl_list)):
        regression_fama_french(merge_df(ew_eig_ret_mtl_list[i],factors_list[i]), f'EW-Eigen-Portfolio: {dates[i][0]} - {dates[i][1]}')
        regression_fama_french(merge_df(ew_SP500_ret_mtl_list[i],factors_list[i]), f'EW- Index: {dates[i][0]} - {dates[i][1]}')
        
        

In [7]:
# Chosing Dates
#dates=[["1991-06-01", "2001-06-01"],["2001-06-01", "2011-06-01"],["2011-06-01", "2021-06-01"]]
dates=[["2010-01-01", "2021-06-01"]]
tickers=['S&P 500', 'DAX']

# Generating Eigenportfolios and performing Fama French
for j in tickers:
    eigenportfolios=[]
    for i in dates:
        eigenportfolios.append(eigenportfolio(i[0],i[1],j))

    # Performing Fama French Analysis
    fama_french(eigenportfolios)

[*********************100%***********************]  499 of 499 completed

3 Failed downloads:
- JEC: No data found, symbol may be delisted
- RTN: No data found, symbol may be delisted
- SSE: None
EW-Eigen-Portfolio: 2010-01-01 - 2021-06-01
                            OLS Regression Results                            
Dep. Variable:                  EW-RF   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     21.06
Date:                Wed, 24 Nov 2021   Prob (F-statistic):           1.55e-06
Time:                        08:57:29   Log-Likelihood:                 59.591
No. Observations:                  25   AIC:                            -111.2
Df Residuals:                      21   BIC:                            -106.3
Df Model:                           3                                         
Covariance Type:            nonrobust            

In [8]:
## Several companys from today dont exist 30 years ago and vice versa,therefore Fama French gets less reliable

## Plotting

In [9]:
# Quant Stats
#for i in range(0,len(ew_eig_ret_list)):
    #qs.reports.basic(ew_eig_ret_list[i])